In [ ]:
#| default_exp xml

# XML

> Concise generation of XML.

In [ ]:
#| export
from fastcore.utils import *

import types,json

from dataclasses import dataclass, asdict
from typing import Mapping
from functools import partial
from html import escape

In [ ]:
from IPython.display import Markdown
from pprint import pprint

In [ ]:
#| export
def _attrmap(o):
    o = dict(htmlClass='class', cls='class', _class='class', klass='class',
             _for='for', fr='for', htmlFor='for').get(o, o)
    return o.lstrip('_').replace('_', '-')

In [ ]:
#|export
class XT(list):
    def __init__(self, tag, cs, attrs=None, void_=False, **kwargs):
        super().__init__([tag, cs, {**(attrs or {}), **kwargs}])
        self.void_ = void_

    @property
    def tag(self): return self[0]
    @property
    def children(self): return self[1]
    @property
    def attrs(self): return self[2]

    def __setattr__(self, k, v):
        if k.startswith('__') or k in ('tag','cs','attrs','void_'): return super().__setattr__(k,v)
        self.attrs[k.lstrip('_').replace('_', '-')] = v

    def __getattr__(self, k):
        if k.startswith('__') or k not in self.attrs: raise AttributeError(k)
        return self.attrs[k.lstrip('_').replace('_', '-')]

In [ ]:
#| export
def xt(tag:str, *c, void_=False, **kw):
    "Create an XML tag structure `[tag,children,attrs]` for `toxml()`"
    if len(c)==1 and isinstance(c[0], types.GeneratorType): c = tuple(c[0])
    kw = {_attrmap(k):v for k,v in kw.items() if v is not None}
    return XT(tag.lower(),c,kw, void_=void_)

In [ ]:
#| export
_g = globals()
_all_ = ['Html', 'Head', 'Title', 'Meta', 'Link', 'Style', 'Body', 'Pre', 'Code',
    'Div', 'Span', 'P', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'Strong', 'Em', 'B',
    'I', 'U', 'S', 'Strike', 'Sub', 'Sup', 'Hr', 'Br', 'Img', 'A', 'Link', 'Nav',
    'Ul', 'Ol', 'Li', 'Dl', 'Dt', 'Dd', 'Table', 'Thead', 'Tbody', 'Tfoot', 'Tr',
    'Th', 'Td', 'Caption', 'Col', 'Colgroup', 'Form', 'Input', 'Textarea',
    'Button', 'Select', 'Option', 'Label', 'Fieldset', 'Legend', 'Details',
    'Summary', 'Main', 'Header', 'Footer', 'Section', 'Article', 'Aside', 'Figure',
    'Figcaption', 'Mark', 'Small', 'Iframe', 'Object', 'Embed', 'Param', 'Video',
    'Audio', 'Source', 'Canvas', 'Svg', 'Math', 'Script', 'Noscript', 'Template', 'Slot']

for o in _all_: _g[o] = partial(xt, o.lower())

The main HTML tags are exported as `xt` partials.

Attributes are passed as keywords. Use 'klass' and 'fr' instead of 'class' and 'for', to avoid Python reserved word clashes.

In [ ]:
samp = Html(
    Head(Title('Some page')),
    Body(Div('Some text', Input(name='me'), Img(src="filename", data=1), klass='myclass'))
)
pprint(samp)

['html',
 (['head', (['title', ('Some page',), {}],), {}],
  ['body',
   (['div',
     ('Some text',
      ['input', (), {'name': 'me'}],
      ['img', (), {'data': 1, 'src': 'filename'}]),
     {'class': 'myclass'}],),
   {}]),
 {}]


The three elements of the list can also be accessed with property names, so you don't have to remember their order.

In [ ]:
elem = P('Some text', id="myid")
print(elem.tag)
print(elem.children)
print(elem.attrs)

p
('Some text',)
{'id': 'myid'}


You can also get and set attrs directly:

In [ ]:
elem.id = 'newid'
print(elem.id)
elem

newid


['p', ('Some text',), {'id': 'newid'}]

In [ ]:
#| export
def _escape(s): return '' if s is None else escape(s) if isinstance(s, str) else s

In [ ]:
#| export
def _to_attr(k,v):
    if isinstance(v,bool):
        if v==True : return str(k)
        if v==False: return ''
    if isinstance(v,str): v = escape(v, quote=True)
    elif isinstance(v, Mapping): v = json.dumps(v)
    else: v = str(v)
    qt = '"'
    if qt in v: qt = "'"
    return f'{k}={qt}{v}{qt}'

In [ ]:
#| export
def to_xml(elm, lvl=0):
    "Convert `xt` element tree into an XML string"
    if elm is None: return ''
    if isinstance(elm, tuple): return '\n'.join(to_xml(o) for o in elm)
    if hasattr(elm, '__xt__'): elm = elm.__xt__()
    sp = ' ' * lvl
    if not isinstance(elm, list): return f'{_escape(elm)}\n'

    tag,cs,attrs = elm
    stag = tag
    if attrs:
        sattrs = (_to_attr(k,v) for k,v in attrs.items())
        stag += ' ' + ' '.join(sattrs)

    isvoid = getattr(elm, 'void_', False)
    cltag = '' if isvoid else f'</{tag}>'
    if not cs: return f'{sp}<{stag}>{cltag}\n'
    if len(cs)==1 and not isinstance(cs[0],(list,tuple)) and not hasattr(cs[0],'__xt__'):
        return f'{sp}<{stag}>{_escape(cs[0])}{cltag}\n'
    res = f'{sp}<{stag}>\n'
    res += ''.join(to_xml(c, lvl=lvl+2) for c in cs)
    if not isvoid: res += f'{sp}{cltag}\n'
    return res

In [ ]:
h = to_xml(samp)
print(h)

<html>
  <head>
    <title>Some page</title>
  </head>
  <body>
    <div class="myclass">
Some text
      <input name="me"></input>
      <img src="filename" data="1"></img>
    </div>
  </body>
</html>



In [ ]:
#| export
def highlight(s, lang='xml'):
    "Markdown to syntax-highlight `s` in language `lang`"
    return f'```{lang}\n{to_xml(s)}\n```'

In [ ]:
#| export
def showtags(s):
    return f"""<code><pre>
{escape(to_xml(s))}
</code></pre>"""

XT._repr_markdown_ = highlight

In [ ]:
#| export
def __getattr__(tag):
    if tag.startswith('_') or tag[0].islower(): raise AttributeError
    def _f(*c, target_id=None, **kwargs): return xt(tag, *c, target_id=target_id, **kwargs)
    return _f

# Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()